In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Building Intelligent Recommender Systems with Merlin

This notebook is created using the latest stable [merlin-tensorflow-inference](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow-inference/tags) container. 

### Overview

Recommender Systems (RecSys) are the engine of the modern internet and the catalyst for human decisions. Building a recommendation system is challenging because it requires multiple stages (data preprocessing, offline training, item retrieval, filtering, ranking, ordering, etc.) to work together seamlessly and efficiently. The biggest challenges for new practitioners are the lack of understanding around what RecSys look like in the real world, and the gap between examples of simple models and a production-ready end-to-end recommender systems.

The figure below represents a four-stage recommender systems. This is more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems.

![fourstage](../images/fourstages.png)

In these series of notebooks, we are going to showcase how we can deploy a four-stage recommender systems using Merlin Systems library easily on [Triton Inference Server](https://github.com/triton-inference-server/server). Let's go over the concepts in the figure briefly. 
- **Retrieval:** This is the step to narrow down millions of items into thousands of candidates. We are going to train a Two-Tower item retrieval model to retrieve the relevant top-K candidate items.
- **Filtering:** This step is to exclude the already interacted  or undesirable items from the candidate items set or to apply business logic rules. Although this is an important step, for this example we skip this step.
- **Scoring:** This is also known as ranking. Here the retrieved and filtered candidate items are being scored. We are going to train a ranking model to be able to use at our scoring step. 
- **Ordering:** At this stage, we can order the final set of items that we want to recommend to the user. Here, we’re able to align the output of the model with business needs, constraints, or criteria.

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=7) at KDD'21 and read more [in this blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

### Learning objectives
- Understanding four stages of recommender systems
- Training retrieval and ranking models with Merlin Models
- Setting up feature store and approximate nearest neighbours (ANN) search libraries
- Deploying trained models to Triton Inference Server with Merlin Systems

In addition to NVIDIA Merlin libraries and the Triton Inference Server client library, we use two external libraries in these series of examples:

- [Feast](https://docs.feast.dev/): an end-to-end open source feature store library for machine learning
- [Faiss](https://github.com/facebookresearch/faiss): a library for efficient similarity search and clustering of dense vectors

You can find more information about `Feast feature store` and `Faiss` libraries in the next notebook.

### Import required libraries and functions

**Compatibility:**

These notebooks are developed and tested using our latest `merlin-tensorflow:22.XX` container on [NVIDIA's docker registry](https://catalog.ngc.nvidia.com/containers?filters=&orderBy=dateModifiedDESC&query=merlin).

In [2]:
# for running this example on GPU, install the following libraries
# %pip install tensorflow "feast<0.20" faiss-gpu

# for running this example on CPU, uncomment the following lines
# %pip install tensorflow-cpu "feast<0.20" faiss-cpu
# %pip uninstall cudf


In [ ]:
import os

from nvtabular.ops import (
    Categorify,
    TagAsUserID,
    TagAsItemID,
    TagAsItemFeatures,
    TagAsUserFeatures,
    AddMetadata,
    Filter,
    Rename
)

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from merlin.datasets.ecommerce import transform_aliccp
import tensorflow as tf

# for running this example on CPU, comment out the line below
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

In [4]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

In this example notebook, we will generate the synthetic train and test datasets mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset to build our recommender system models.

First, we define our input path and feature repo path.

In [5]:
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
# set up the base dir for feature store
BASE_DIR = os.environ.get(
    "BASE_DIR", "/Merlin/examples/Building-and-deploying-multi-stage-RecSys/"
)

Then, we use `generate_data` utility function to generate synthetic dataset. 

In [6]:
from merlin.datasets.synthetic import generate_data

NUM_ROWS = os.environ.get("NUM_ROWS", 100_000)
train_raw, valid_raw = generate_data("aliccp-raw", int(NUM_ROWS), set_sizes=(0.7, 0.3))

If you would like to use the real ALI-CCP dataset, you can use [get_aliccp()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/datasets/ecommerce/aliccp/dataset.py) function instead. This function takes the raw csv files, and generate parquet files that can be directly fed to NVTabular workflow above.

### Training a Retrieval Model with Two-Tower Model

We start with the offline candidate retrieval stage. We are going to train a Two-Tower model for item retrieval. To learn more about the Two-tower model you can visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb).

#### Feature Engineering with NVTabular

We are going to process our raw categorical features by encoding them using `Categorify()` operator and tag the features with `user` or `item` tags in the schema file. To learn more about [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) and the schema object visit this example [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) in the Merlin Models repo.

In [7]:
output_path = os.path.join(DATA_FOLDER, "processed/retrieval")

We select only positive interaction rows where `click==1` in the dataset with `Filter()` operator.

In [8]:
user_id = ["user_id"] >> Categorify(dtype="int32", out_path='./categories_tt') >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32", out_path='./categories_tt') >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32", out_path='./categories_tt') >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ] >> Categorify(dtype="int32", out_path='./categories_tt') >> TagAsUserFeatures()
)

inputs = user_id + item_id + item_features + user_features + ["click"]

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

transform_aliccp(
    (train_raw, valid_raw),
    output_path,
    nvt_workflow=outputs,
    workflow_name="workflow_retrieval",
)

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


NVTabular exported the schema file, `schema.pbtxt` a protobuf text file, of our processed dataset. To learn more about the schema object and schema file you can explore [02-Merlin-Models-and-NVTabular-integration.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) notebook.

In [9]:
train_tt = Dataset(os.path.join(output_path, "train", "*.parquet"))
valid_tt = Dataset(os.path.join(output_path, "valid", "*.parquet"))

schema = train_tt.schema
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

In [10]:
model_tt = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

2022-07-29 16:05:31.324620: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [11]:
model_tt.compile(
    optimizer="adam",
    run_eagerly=False,
    loss="categorical_crossentropy",
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)],
)
model_tt.fit(train_tt, validation_data=valid_tt, batch_size=1024 * 8, epochs=1)

5/5 [==============================] - 10s 444ms/step - loss: 8.9095 - recall_at_10: 0.0059 - ndcg_at_10: 0.0032 - regularization_loss: 0.0000e+00 - val_loss: 8.9096 - val_recall_at_10: 0.0079 - val_ndcg_at_10: 0.0036 - val_regularization_loss: 0.0000e+00


### Exporting query (user) model

We export the query tower to use it later during the model deployment stage with Merlin Systems.

In [12]:
query_tower = model_tt.retrieval_block.query_block()
query_tower.save(os.path.join(BASE_DIR, "query_tower"))

### Training a Ranking Model with DLRM

Now we will move onto training an offline ranking model. This ranking model will be used for scoring our retrieved items.

#### Feature Engineering with NVTabular

Define output path.

In [13]:
output_path = os.path.join(DATA_FOLDER, "processed/ranking")

In [14]:
%%time

user_id = ["user_id"] >> Categorify(dtype="int32") >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32") >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32") >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ] >> Categorify(dtype="int32") >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features + targets

CPU times: user 162 µs, sys: 17 µs, total: 179 µs
Wall time: 183 µs


Let's call `transform_aliccp` utility function to be able to perform `fit` and `transform` steps on the raw dataset applying the operators defined in the NVTabular workflow pipeline below, and also save our workflow model. After fit and transform, the processed parquet files are saved to output_path.

In [15]:
transform_aliccp(
    (train_raw, valid_raw), output_path, nvt_workflow=outputs, workflow_name="workflow_ranking"
)

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


We use the `schema` object to define our model.

In [16]:
# define train and valid dataset objects
train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

# define schema object
schema = train.schema

In [17]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture is a popular neural network model originally proposed by Facebook in 2019. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074). To learn more about DLRM architetcture please visit `Exploring-different-models` [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/04-Exporting-ranking-models.ipynb) in the Merlin Models GH repo.

In [18]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [19]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=16 * 1024)

5/5 [==============================] - 4s 231ms/step - loss: 0.6932 - auc: 0.4999 - regularization_loss: 0.0000e+00 - val_loss: 0.6931 - val_auc: 0.5002 - val_regularization_loss: 0.0000e+00


Let's save our DLRM model to be able to load back at the deployment stage. 

In [20]:
model.save(os.path.join(BASE_DIR, "dlrm"))

In the following cells we are going to export the required user and item features files, and save the query (user) tower model and item embeddings to disk. If you want to read more about exporting retrieval models, please visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb) notebook in Merlin Models library repo.

### Set up a feature store with Feast

Before we move onto the next step, we need to create a Feast feature repository. We will create the feature repo in the current working directory, which is `BASE_DIR` for us.

In [21]:
!rm -rf $BASE_DIR/feature_repo
!cd $BASE_DIR && feast init feature_repo


Creating a new Feast repository in /Merlin/examples/Building-and-deploying-multi-stage-RecSys/feature_repo.



You should be seeing a message like <i>Creating a new Feast repository in ... </i> printed out above. Now, navigate to the `feature_repo` folder and remove the demo parquet file created by default, and `examples.py` file.

In [22]:
feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
if os.path.exists(f"{feature_repo_path}/example.py"):
    os.remove(f"{feature_repo_path}/example.py")
if os.path.exists(f"{feature_repo_path}/data/driver_stats.parquet"):
    os.remove(f"{feature_repo_path}/data/driver_stats.parquet")

### Exporting user and item features

In [23]:
from merlin.models.utils.dataset import unique_rows_by_features

user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID)
    .compute()
    .reset_index(drop=True)
)

In [24]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1,1,1,1,1,1,1,2,2,2
2,3,3,1,1,1,1,1,1,1,3,3,3
3,4,4,1,1,1,1,1,1,1,4,4,4
4,5,5,1,1,1,1,1,1,1,5,5,5


We will artificially add `datetime` and `created` timestamp columns to our user_features dataframe. This required by Feast to track the user-item features and their creation time and to determine which version to use when we query Feast.

In [25]:
from datetime import datetime

user_features["datetime"] = datetime.now()
user_features["datetime"] = user_features["datetime"].astype("datetime64[ns]")
user_features["created"] = datetime.now()
user_features["created"] = user_features["created"].astype("datetime64[ns]")

In [26]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,datetime,created
0,1,1,1,1,1,1,1,1,1,1,1,1,2022-07-29 16:06:09.642304,2022-07-29 16:06:09.643715
1,2,2,1,1,1,1,1,1,1,2,2,2,2022-07-29 16:06:09.642304,2022-07-29 16:06:09.643715
2,3,3,1,1,1,1,1,1,1,3,3,3,2022-07-29 16:06:09.642304,2022-07-29 16:06:09.643715
3,4,4,1,1,1,1,1,1,1,4,4,4,2022-07-29 16:06:09.642304,2022-07-29 16:06:09.643715
4,5,5,1,1,1,1,1,1,1,5,5,5,2022-07-29 16:06:09.642304,2022-07-29 16:06:09.643715


In [27]:
user_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "user_features.parquet")
)

In [28]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
    .compute()
    .reset_index(drop=True)
)

In [29]:
item_features.shape

(451, 4)

In [30]:
item_features["datetime"] = datetime.now()
item_features["datetime"] = item_features["datetime"].astype("datetime64[ns]")
item_features["created"] = datetime.now()
item_features["created"] = item_features["created"].astype("datetime64[ns]")

In [31]:
item_features.head()

,item_id,item_category,item_shop,item_brand,datetime,created
0,1,1,1,1,2022-07-29 16:06:09.742965,2022-07-29 16:06:09.745334
1,2,2,2,2,2022-07-29 16:06:09.742965,2022-07-29 16:06:09.745334
2,3,3,3,3,2022-07-29 16:06:09.742965,2022-07-29 16:06:09.745334
3,4,4,4,4,2022-07-29 16:06:09.742965,2022-07-29 16:06:09.745334
4,5,5,5,5,2022-07-29 16:06:09.742965,2022-07-29 16:06:09.745334


In [32]:
# save to disk
item_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "item_features.parquet")
)

### Extract and save Item embeddings

In [33]:
item_embs = model_tt.item_embeddings(
    Dataset(item_features, schema=schema), batch_size=1024
)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [34]:
# select only item_id together with embedding columns
item_embeddings = item_embs_df.drop(
    columns=["item_category", "item_shop", "item_brand"]
)

In [35]:
item_embeddings.head()

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,0.021452,-0.039451,-0.027525,0.004748,-0.038861,0.004086,-0.015699,-0.043150,0.009985,...,0.004878,-0.017287,-0.052191,-0.039028,0.009816,0.030303,-0.023015,0.025190,-0.048255,-0.004642
1,2,-0.006522,-0.024186,-0.014698,0.026402,-0.038433,-0.005294,-0.022629,0.008153,0.027704,...,-0.003684,-0.002784,-0.006920,-0.008715,-0.046778,0.010102,-0.056791,0.017473,-0.028276,-0.014962
2,3,0.015643,-0.039508,-0.008537,-0.006320,-0.047506,-0.041426,0.011790,-0.028635,0.022870,...,0.004367,-0.024234,-0.035562,-0.031195,0.067566,0.044429,0.007656,0.018322,-0.010219,-0.003382
3,4,0.035367,-0.004361,-0.004915,-0.007458,-0.024948,-0.022634,-0.008486,0.002527,0.020076,...,0.040158,-0.015545,-0.023736,0.010247,0.027722,0.016833,0.007536,0.004979,-0.018067,-0.008719
4,5,0.037872,-0.031143,0.030403,0.025478,-0.051041,0.031561,0.002363,-0.020428,0.011938,...,-0.003469,-0.013381,0.009062,-0.024175,-0.010814,0.011117,-0.011316,-0.014056,-0.036326,-0.001744


In [36]:
# save to disk
item_embeddings.to_parquet(os.path.join(BASE_DIR, "item_embeddings.parquet"))

### Create feature definitions 

Now we will create our user and item features definitions in the user_features.py and item_features.py files and save these files in the feature_repo.

In [37]:
file = open(os.path.join(BASE_DIR, "feature_repo/", "user_features.py"), "w")
file.write(
    """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

user_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

user = Entity(name="user_id", value_type=ValueType.INT32, description="user id",)

user_features_view = FeatureView(
    name="user_features",
    entities=["user_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="user_shops", dtype=ValueType.INT32),
        Feature(name="user_profile", dtype=ValueType.INT32),
        Feature(name="user_group", dtype=ValueType.INT32),
        Feature(name="user_gender", dtype=ValueType.INT32),
        Feature(name="user_age", dtype=ValueType.INT32),
        Feature(name="user_consumption_2", dtype=ValueType.INT32),
        Feature(name="user_is_occupied", dtype=ValueType.INT32),
        Feature(name="user_geography", dtype=ValueType.INT32),
        Feature(name="user_intentions", dtype=ValueType.INT32),
        Feature(name="user_brands", dtype=ValueType.INT32),
        Feature(name="user_categories", dtype=ValueType.INT32),
    ],
    online=True,
    input=user_features,
    tags=dict(),
)
""".format(
        os.path.join(BASE_DIR, "feature_repo/data/", "user_features.parquet")
    )
)
file.close()

In [38]:
with open(os.path.join(BASE_DIR, "feature_repo/", "item_features.py"), "w") as f:
    f.write(
        """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

item_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

item = Entity(name="item_id", value_type=ValueType.INT32, description="item id",)

item_features_view = FeatureView(
    name="item_features",
    entities=["item_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="item_category", dtype=ValueType.INT32),
        Feature(name="item_shop", dtype=ValueType.INT32),
        Feature(name="item_brand", dtype=ValueType.INT32),
    ],
    online=True,
    input=item_features,
    tags=dict(),
)
""".format(
            os.path.join(BASE_DIR, "feature_repo/data/", "item_features.parquet")
        )
    )
file.close()

Let's checkout our Feast feature repository structure.

In [39]:
# install seedir
!pip install seedir

In [40]:
import seedir as sd

feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
sd.seedir(
    feature_repo_path,
    style="lines",
    itemlimit=10,
    depthlimit=3,
    exclude_folders=".ipynb_checkpoints",
    sort=True,
)

feature_repo/
├─__init__.py
├─data/
│ ├─item_features.parquet
│ └─user_features.parquet
├─feature_store.yaml
├─item_features.py
└─user_features.py


### Next Steps
We trained and exported our ranking and retrieval models and NVTabular workflows. In the next step, we will learn how to deploy our trained models into [Triton Inference Server (TIS)](https://github.com/triton-inference-server/server) with Merlin Systems library.

For the next step, move on to the `02-Deploying-multi-stage-Recsys-with-Merlin-Systems.ipynb` notebook to deploy our saved models as an ensemble to TIS and obtain prediction results for a given request.